In [1]:
import torch
import torchvision
# import
import os
import pickle
import numpy as np
import PIL.Image

import scipy.io as sio
from datetime import datetime
import sys


sys.path.append('../model')
from C3D import C3D
                

sys.path.append('../cnn_preferred')
from utils import normalise_img, clip_extreme_pixel, save_video, normalise_vid, get_cnn_features
from activation_maximization import generate_preferred

In [2]:
os.path.isdir('../cnn_preferred')

True

In [14]:
net = C3D.C3D()
param_file = os.path.join('../model','C3D', 'c3d.pickle')
net.load_state_dict(torch.load(param_file))
net.eval()

C3D(
  (conv1): Conv3d(3, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool1): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool2): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3a): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv3b): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool3): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv4a): Conv3d(256, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv4b): Conv3d(512, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv5a): Conv3d(512, 512, kernel_size=(3, 3, 3), stride=

In [8]:
net = torchvision.models.alexnet(pretrained=True)

In [4]:
net = torchvision.models.resnet50(pretrained=True)
#net = torchvision.models.vgg16(pretrained=True)

In [15]:
img_mean = np.float32([104, 117, 123])
img_std = np.float32([1,1,1])

# image
#img_mean=np.array([0.485, 0.456, 0.406],dtype=np.float),
#img_std = np.array([0.229,0.224,0.225])

In [16]:
#save_dir
save_dir = '../result'
save_folder = 'jupyter_demo_torch_C3D_TV_norm_all'#__file__.split('.')[0]
save_folder = save_folder + '_' + datetime.now().strftime('%Y%m%dT%H%M%S')
save_path = os.path.join(save_dir,save_folder)
os.makedirs(save_path, exist_ok=True)

In [5]:
def hook(module, input, output):
    outputs.append(output.clone())

In [6]:
net

C3D(
  (conv1): Conv3d(3, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool1): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool2): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3a): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv3b): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool3): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv4a): Conv3d(256, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv4b): Conv3d(512, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv5a): Conv3d(512, 512, kernel_size=(3, 3, 3), stride=

In [23]:
layer_list = ['fc8']
layer_list = ['conv4a']
layer = layer_list[0]

In [24]:
# initial image for the optimization
fr, h, w = 16, 112,112
initial_video = np.zeros((fr, h,w,3),dtype='float32')

h, w = 224,224
initial_image = np.zeros(( h,w,3),dtype='float32')


initial_input = np.random.randint(0, 256, (h,w,3))

In [25]:
# video

#targetlayer
exec_str_list = ["net."+layer+'.register_forward_hook(hook)']

exec("num_of_ch = net."+layer+".weight.detach().numpy().shape[0]") #param_list[layer_value].shape[0]
num_of_img = 10
step = int(num_of_ch/num_of_img)
channel_list = range(0,num_of_ch,step)

ee = get_cnn_features(net,torch.Tensor(initial_video.transpose(3,0, 1, 2)[np.newaxis]), exec_str_list)
feat_num = ee[0].detach().numpy().shape

1


In [13]:
ee

[tensor([[ 1.8620, -1.6925, -0.6393, -0.2916,  0.4595, -0.4594, -0.4528,
           0.2161,  0.7801, -0.4271,  0.9405,  0.2597,  1.0147, -0.5320,
          -0.3751,  0.7027,  0.5555, -0.1977,  0.0808,  0.4149, -0.0149,
          -0.5480, -0.8084, -0.4998, -0.2819, -0.9104, -0.2909, -0.9761,
          -0.7457, -1.6968, -0.6902, -1.0913, -1.2734, -0.8816, -0.6625,
          -1.5463, -1.3037, -1.2240, -1.3473, -0.2895, -0.3981, -0.3294,
          -0.1359, -0.0137,  0.8445, -0.1700, -0.2184, -0.6449, -0.5671,
          -0.6598, -0.6680, -0.8099, -0.5671, -0.7802, -0.4564, -1.1022,
          -0.4305,  0.6165, -1.1533,  0.3974, -0.2420, -0.2280, -0.0916,
          -0.5005, -0.9549, -1.0626, -0.4695, -1.0169, -0.4525, -0.5529,
          -1.7205, -1.7023, -0.7665, -0.4090, -0.4559, -0.4317, -0.3563,
           0.8303,  0.0300,  0.3464,  0.2805,  0.2395,  0.3012,  0.3790,
           1.8192,  1.1401,  0.4927, -0.9365, -0.8516, -0.7921, -0.8787,
          -0.6536, -0.9242, -1.3293, -0.7037, -1.18

In [ ]:
#targetlayer
exec_str_list = ["net.classifier[6].register_forward_hook(hook)"]
exec(exec_str_list[0])
exec("num_of_ch = net.classifier[6].weight.detach().numpy().shape[0]") #param_list[layer_value].shape[0]
num_of_img = 10
step = int(num_of_ch/num_of_img)
channel_list = range(1,num_of_ch,step)
outputs = []
ee = net(torch.Tensor(initial_image.transpose(2,0, 1)[np.newaxis]))
feat_num = outputs[0].detach().numpy().shape

In [ ]:
#targetlayer
exec_str_list = ["net.conv1.register_forward_hook(hook)"]
exec(exec_str_list[0])
exec("num_of_ch = net.conv1.weight.detach().numpy().shape[0]") #param_list[layer_value].shape[0]
num_of_img = 10
step = int(num_of_ch/num_of_img)
channel_list = range(1,num_of_ch,step)
outputs = []
ee = net(torch.Tensor(initial_image.transpose(2,0, 1)[np.newaxis]))
feat_num = outputs[0].detach().numpy().shape

In [ ]:
net

In [7]:
target_layer = "net.layer1[2].conv3"
target_layer = "net.layer4[0].bn3"

In [8]:
#targetlayer
exec_str_list = [target_layer +".register_forward_hook(hook)"]


exec("num_of_ch = "+target_layer+".weight.detach().numpy().shape[0]") #param_list[layer_value].shape[0]
num_of_img = 10
step = int(num_of_ch/num_of_img)
channel_list = range(0,num_of_ch,step)

ee = get_cnn_features(net,torch.Tensor(initial_input.transpose(2,0, 1)[np.newaxis]), exec_str_list)
feat_num = ee[0].detach().numpy().shape

1


In [9]:
feat_num

(1, 2048, 7, 7)

In [36]:
# options
opts = {
    'img_mean': img_mean,
    'img_std': img_std,
    'exec_code': exec_str_list, # exection code inside the function

    'iter_n': 200, # the total number of iterations for gradient descend

    'disp_every': 1, # display the information on the terminal for every n iterations

    'save_intermediate': True, # save the intermediate or not
    'save_intermediate_every': 10, # save the intermediate for every n iterations
    'save_intermediate_path': save_path, # the path to save the intermediate

    'lr_start': 1., # learning rate
    'lr_end': 1.,

    'momentum_start': 0.001, # gradient with momentum
    'momentum_end': 0.001,

    'decay_start': 0.001, # pixel decay for each iteration
    'decay_end': 0.001,

    'image_blur': True, # Use image smoothing or not
    'sigma_start': 2.5, # the size of the gaussian filter for image smoothing
    'sigma_end': 0.5,

    'image_jitter': True, # use image jittering during
    'jitter_size': 4,
    
    'use_p_norm_reg': False,
    'p': 2,

    'use_TV_norm_reg': False,
    'TVbeta1': 1, 
    'TVbeta2':1.2,

    'clip_small_norm': True,
    'clip_small_norm_every': 1,
    'n_pct_start': 5,
    'n_pct_end': 5,

    'clip_small_contribution': True,
    'clip_small_contribution_every': 1,
    'c_pct_start': 5,
    'c_pct_end':5,
    
    'input_size': (16, 112,112,3),
    #'initial_input': None, # the initial image for the optimization (setting to None will use random noise as initial image)
    'initial_input': initial_video,
    }




In [20]:
# options
opts = {
    'img_mean': img_mean,
    'img_std': img_std,
    'exec_code': exec_str_list, # exection code inside the function

    'iter_n': 200, # the total number of iterations for gradient descend

    'disp_every': 1, # display the information on the terminal for every n iterations

    'save_intermediate': True, # save the intermediate or not
    'save_intermediate_every': 10, # save the intermediate for every n iterations
    'save_intermediate_path': save_path, # the path to save the intermediate

    'lr_start': 1., # learning rate
    'lr_end': 1.,

    'momentum_start': 0.001, # gradient with momentum
    'momentum_end': 0.001,

    'decay_start': 0.001, # pixel decay for each iteration
    'decay_end': 0.001,

    'image_blur': True, # Use image smoothing or not
    'sigma_start': 2.5, # the size of the gaussian filter for image smoothing
    'sigma_end': 0.5,

    'image_jitter': True, # use image jittering during
    'jitter_size': 4,
    
    'use_p_norm_reg': False,
    'p': 2,

    'use_TV_norm_reg': False,
    'TVbeta1': 1, 
    'TVbeta2':1.2,

    'clip_small_norm': True,
    'clip_small_norm_every': 1,
    'n_pct_start': 5,
    'n_pct_end': 5,

    'clip_small_contribution': True,
    'clip_small_contribution_every': 1,
    'c_pct_start': 5,
    'c_pct_end':5,
    
    #'input_size': (16, 112,112,3),
    'input_size': (224,224,3),
    #'initial_input': None, # the initial image for the optimization (setting to None will use random noise as initial image)
    'initial_input': initial_input,
    }



In [10]:
# options for resnet
opts = {
    'img_mean': img_mean,
    'img_std': img_std,
    'exec_code': exec_str_list, # exection code inside the function

    'iter_n': 200, # the total number of iterations for gradient descend

    'disp_every': 1, # display the information on the terminal for every n iterations

    'save_intermediate': True, # save the intermediate or not
    'save_intermediate_every': 10, # save the intermediate for every n iterations
    'save_intermediate_path': save_path, # the path to save the intermediate

    'lr_start': 0.01, # learning rate
    'lr_end': 0.01,

    'momentum_start': 0.00001, # gradient with momentum
    'momentum_end': 0.00001,

    'decay_start': 0.001, # pixel decay for each iteration
    'decay_end': 0.001,

    'image_blur': True, # Use image smoothing or not
    'sigma_start': 2.5, # the size of the gaussian filter for image smoothing
    'sigma_end': 0.5,

    'image_jitter': True, # use image jittering during
    'jitter_size': 32,

    'use_TV_norm_reg': False,
    'TVbeta1': 1, 
    'TVbeta2':1.2,

    'clip_small_norm': True,
    'clip_small_norm_every': 1,
    'n_pct_start': 5,
    'n_pct_end': 5,

    'clip_small_contribution': True,
    'clip_small_contribution_every': 1,
    'c_pct_start': 5,
    'c_pct_end':5,
    
    #'input_size': (16, 112,112,3),
    'input_size': (224,224,3),
    #'initial_input': None, # the initial image for the optimization (setting to None will use random noise as initial image)
    'initial_input': initial_input,
    }



In [11]:
channel_list = [149]

In [12]:
for channel in channel_list:
    #
    print('')
    print('channel='+str(channel))
    print('')

    # target units
    feat_size = feat_num
    y_index = int(feat_size[2]/2) # the unit in the center of feature map
    x_index = int(feat_size[3]/2) # the unit in the center of feature map
    feature_mask = np.zeros(feat_size)
    feature_mask[0,channel,y_index,x_index] = 1

    # weights for the target units
    feature_weight = np.zeros(feat_size, dtype=np.float32)
    feature_weight[:] = 1.
    #
    preferred_vid = generate_preferred(net, feature_mask, feature_weight=feature_weight, **opts)

    # save the results
    save_name = 'preferred_img' + '_layer_' + str(layer) + '_channel_' + str(channel) + '.mat'
    sio.savemat(os.path.join(save_path,save_name),{'preferred_vid':preferred_vid})

    save_name = 'preferred_img' + '_layer_' + str(layer) + '_channel_' + str(channel) + '.avi'
    save_video(normalise_vid(clip_extreme_pixel(preferred_vid,pct=0.04)), save_name, save_path )


channel=149

1
iter=1; mean(abs(feat))=0.297181;
1
iter=2; mean(abs(feat))=0.258261;
1
iter=3; mean(abs(feat))=0.133296;
1
iter=4; mean(abs(feat))=0.167845;
1
iter=5; mean(abs(feat))=0.361009;
1
iter=6; mean(abs(feat))=1.20787;
1
iter=7; mean(abs(feat))=0.225;
1
iter=8; mean(abs(feat))=0.899251;
1
iter=9; mean(abs(feat))=0.682899;
1
iter=10; mean(abs(feat))=0.245953;
1
iter=11; mean(abs(feat))=0.402756;
1
iter=12; mean(abs(feat))=0.10336;
1
iter=13; mean(abs(feat))=0.198953;
1
iter=14; mean(abs(feat))=0.114092;
1
iter=15; mean(abs(feat))=0.445952;
1
iter=16; mean(abs(feat))=0.412937;
1
iter=17; mean(abs(feat))=0.239378;
1
iter=18; mean(abs(feat))=0.20278;
1
iter=19; mean(abs(feat))=0.210112;
1
iter=20; mean(abs(feat))=0.241979;
1
iter=21; mean(abs(feat))=0.00748792;
1
iter=22; mean(abs(feat))=0.755881;
1
iter=23; mean(abs(feat))=0.744542;
1
iter=24; mean(abs(feat))=0.124209;
1
iter=25; mean(abs(feat))=0.287637;
1
iter=26; mean(abs(feat))=0.591664;
1
iter=27; mean(abs(feat))=0.774429;


NameError: name 'layer' is not defined

In [28]:
channel_list = [23]

In [37]:
# video
for channel in channel_list:
    #
    print('')
    print('channel='+str(channel))
    print('')

    # target units
    feat_size = feat_num
    t_index = int(feat_size[2]/2)
    y_index = int(feat_size[3]/2) # the unit in the center of feature map
    x_index = int(feat_size[4]/2) # the unit in the center of feature map
    feature_mask = np.zeros(feat_size)
    feature_mask[0,channel,t_index, y_index,x_index] = 1

    # weights for the target units
    feature_weight = np.zeros(feat_size, dtype=np.float32)
    feature_weight[:] = 1.
    #
    preferred_vid = generate_preferred(net, feature_mask, feature_weight=feature_weight, **opts)

    # save the results
    save_name = 'preferred_img' + '_layer_' + str(layer) + '_channel_' + str(channel) + '.mat'
    sio.savemat(os.path.join(save_path,save_name),{'preferred_vid':preferred_vid})

    save_name = 'preferred_img' + '_layer_' + str(layer) + '_channel_' + str(channel) + '.avi'
    save_video(normalise_vid(clip_extreme_pixel(preferred_vid,pct=0.04)), save_name, save_path )


channel=23

1
iter=1; mean(abs(feat))=4.71797;
1
iter=2; mean(abs(feat))=328.048;
1
iter=3; mean(abs(feat))=627.787;
1
iter=4; mean(abs(feat))=822.265;
1
iter=5; mean(abs(feat))=1149.37;
1
iter=6; mean(abs(feat))=1239.92;
1
iter=7; mean(abs(feat))=1461.47;
1
iter=8; mean(abs(feat))=1448.2;
1
iter=9; mean(abs(feat))=1638.48;
1
iter=10; mean(abs(feat))=1589.6;
1
iter=11; mean(abs(feat))=1647.17;
1
iter=12; mean(abs(feat))=1769.25;
1
iter=13; mean(abs(feat))=1524.04;
1
iter=14; mean(abs(feat))=1660.6;
1
iter=15; mean(abs(feat))=1712.89;
1
iter=16; mean(abs(feat))=1856.9;
1
iter=17; mean(abs(feat))=2035.2;
1
iter=18; mean(abs(feat))=1855.03;
1
iter=19; mean(abs(feat))=2101.78;
1
iter=20; mean(abs(feat))=2024.25;
1
iter=21; mean(abs(feat))=2057.42;
1
iter=22; mean(abs(feat))=2390.24;
1
iter=23; mean(abs(feat))=2378.4;
1
iter=24; mean(abs(feat))=2129.09;
1
iter=25; mean(abs(feat))=2454.77;
1
iter=26; mean(abs(feat))=2369.3;
1
iter=27; mean(abs(feat))=2474.32;
1
iter=28; mean(abs(feat))=2632

KeyboardInterrupt: 

In [35]:
exec_str_list

['net.conv4a.register_forward_hook(hook)']

In [33]:
feature_mask.shape

(1, 512, 4, 14, 14)